Given a list of ids,
If the list is empty, return.
If not, see if they can be all be retrieved. Idf so , return
If the list is length one, ty to retrieve it, add to the list of problems if it fails, and return.
If not, split the list in half and repeat the process.

In [1]:
import logging

from s2agr.builder import Builder
from s2agr.monitor import LoggingMonitor
from s2agr.persistence.database_catalogue import DatabaseCatalogue, production_connection

In [2]:
connection = production_connection()
catalogue = DatabaseCatalogue(connection)
librarian = Builder().with_catalogue(catalogue).with_monitor(LoggingMonitor(log_level=logging.DEBUG)).build()

In [3]:
IN_ID = 'abd1c342495432171beb7ca8fd9551ef13cbd0ff'

In [4]:
in_citer_rows = list(catalogue.query('select citing_id from citation where cited_id = (%s) and is_influential', IN_ID))
in_citer_ids = [row[0] for row in in_citer_rows]

In [12]:
def retrieve_all(ids):
    librarian.get_papers(*ids)


def check_ids(ids, problems:set = None) -> set:
    print('trying ', len(ids))
    if problems is None:
        problems = set()
    else:
        problems = problems
    if len(ids) == 0:
        return problems
    try:
        retrieve_all(ids)
        print('got ', len(ids))
        return problems
    except:
        if len(ids) == 1:
            problems.add(ids[0])
            return problems
        split = len(ids) // 2
        return check_ids(ids[:split]).union(check_ids(ids[split:]))


In [13]:
problems = check_ids(in_citer_ids)

trying  950
{'message': 'Internal Server Error'}
trying  475
{'message': 'Internal Server Error'}
trying  237
{'message': 'Internal Server Error'}
trying  118
got  118
trying  119
{'message': 'Internal Server Error'}
trying  59
{'message': 'Internal Server Error'}
trying  29
got  29
trying  30
got  30
trying  60
{'message': 'Internal Server Error'}
trying  30
{'message': 'Internal Server Error'}
trying  15
{'message': 'Internal Server Error'}
trying  7
got  7
trying  8
{'message': 'Internal Server Error'}
trying  4
got  4
trying  4
{'message': 'Internal Server Error'}
trying  2
{'message': 'Internal Server Error'}
trying  1
got  1
trying  1
{'message': 'Internal Server Error'}
trying  2
got  2
trying  15
got  15
trying  30
{'message': 'Internal Server Error'}
trying  15
got  15
trying  15
{'message': 'Internal Server Error'}
trying  7
got  7
trying  8
{'message': 'Internal Server Error'}
trying  4
got  4
trying  4
{'message': 'Internal Server Error'}
trying  2
got  2
trying  2
{'messag

In [14]:
problems

{'300f6c733ed8a796e39e4906c40bb84e56dec8b1',
 '6d26a7d1d6de855db5b632766835a9f010f64938',
 '75a5515a70f67764dbec22d40d1c574b50851253',
 'b18f37c6681e21b9f9f5e1771b1a5b4f58351869',
 'c0f1ee6c898a0f5a82b7badbbb369346cc070bd8',
 'd026163813a3586bec36364ac8a0944dcb040359',
 'f56745ce16beb2f19689e5008eeccd0de88cf430'}

In [15]:
for id in problems:
    librarian.get_paper(id)

In [16]:
with open('problem_id_list','a') as f:
    for id in problems:
        f.write('%s\n' % id)